##  Imports

In [ ]:
from glob import glob
import os
import numpy as np
import matplotlib.pyplot as plt
import shutil
from torchvision import transforms
from torchvision import models
import torch
from torch.autograd import Variable
import torch.nn as nn
from torch.optim import lr_scheduler
from torch import optim
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
import time
%matplotlib inline

## Check if GPU is present 
Check if GPU is available and if so use it by default, otherwise use CPU only.

In [ ]:
if torch.cuda.is_available():
    is_cuda = True

## Data processing

We are going to create a model to enter the [Dogs vs Cats](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition) competition at Kaggle.

Alternatively, a direct link to the catvsdogs [dataset](http://files.fast.ai/data/dogscats.zip).


## Peep look into the downloaded data 

There are 25,000 labelled dog and cat photos available for training, and 12,500 in the test set that we have to try to label for this competition. According to the Kaggle web-site, when this competition was launched (end of 2013): *"**State of the art**: The current literature suggests machine classifiers can score above 80% accuracy on this task"*. So if you can beat 80%, then you will be at the cutting edge as of 2013!

If you run on cpu, you should use the small number of images.

```   
        DogsCats/
            train/
                dog/
                    dog.183.jpg
                    dog.186.jpg
                    dog.193.jpg
                cat/
                    cat.17.jpg
                    cat.2.jpg
                    cat.27.jpg
            valid/
                dog/
                    dog.173.jpg
                    dog.156.jpg
                    dog.123.jpg
                cat/
                    cat.172.jpg
                    cat.20.jpg
                    cat.21.jpg

```

In [ ]:
simple_transform = transforms.Compose([transforms.Resize((224,224))
                                       ,transforms.ToTensor()
                                       ,transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
train = ImageFolder('DogsCats_small/train/',simple_transform)
valid = ImageFolder('DogsCats_small/valid/',simple_transform)

## Class labels
Labels in this case are assigned automatically

In [ ]:
print(train.class_to_idx)
print(train.classes) 


## Function to show the images

In [ ]:

def imshow(inp):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)

## Show an image

In [ ]:
imshow(train[50][0])

## Create data generators

```datasets``` is a class of the ```torchvision``` package (see [torchvision.datasets](http://pytorch.org/docs/master/torchvision/datasets.html)) and deals with data loading. It integrates a multi-threaded loader that fetches images from the disk, groups them in mini-batches and serves them continously to the GPU right after each _forward_/_backward_ pass through the network. 

In [ ]:
train_data_gen = torch.utils.data.DataLoader(train,shuffle=True,batch_size=64,num_workers=3)
valid_data_gen = torch.utils.data.DataLoader(valid,batch_size=64,num_workers=3)

In [ ]:
dataset_train = len(train_data_gen.dataset)
dataset_valid = len(valid_data_gen.dataset)

## Create a network

In this practical example, our goal is to use the already trained model and just change the number of output classes. To this end we replace the last ```nn.Linear``` layer trained for 1000 classes to ones with 2 classes. We can use any [pre-trained network](https://pytorch.org/docs/stable/torchvision/models.html?highlight=models). In this case, we are opting for resnet with 18 layers, namely, resnet18.

<img width= 700 src='resnet18.png'>

In [ ]:
model_ft = models.resnet18(pretrained=True)
num_ftrs = model_ft.fc.in_features
model_ft.fc = nn.Linear(num_ftrs, 2)

if torch.cuda.is_available():
    model_ft = model_ft.cuda()

## Print the Network


In [ ]:
model_ft

## Training fully connected module

### Creating loss function 

In [ ]:
criterion = nn.CrossEntropyLoss()

### Creating optimizer

In [ ]:
# Loss and Optimizer
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

### train module

 #### Clear gradients:
    There could be gradients from previous batches, therefore it’s necessary to clear gradient after every epoch
  ####  Forward pass: 
    This step computes the predicted outputs by passing inputs to the convolutional neural network model
  #### Calculate loss:
    As the model trains, the loss function calculates the loss after every epoch and then it is used by the optimizer.
  #### Backward pass: 
    This step computes the gradient of the loss with respect to model parameters
  ####  Optimization
    This performs a single optimization step/ parameter update for the model
    Update schedular after each epoch

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=5):
    since = time.time()

    best_model_wts = model.state_dict()
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Set model to training mode
        model.train()  
        
        running_loss = 0.0
        running_corrects = 0

        # Iterate over data.
        for di, data in enumerate(train_data_gen):
            # get the inputs
            inputs, labels = data

            # wrap them in Variable
            if torch.cuda.is_available():
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)

            # zero the parameter gradients
            optimizer.zero_grad()
                
            # forward
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)

                # backward + optimize only if in training phase
            loss.backward()
            optimizer.step()

            # statistics
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds.data == labels.data)
            
        scheduler.step()
            
        epoch_loss = float(running_loss) / dataset_train
        epoch_acc = float(running_corrects) / dataset_train


        print('Train: Loss: {:.4f} Acc: {:.2f}%'.format( epoch_loss, epoch_acc*100))
        print()

         # perform validation
        epoch_acc = test_model(model_ft, 'Validation')
        if epoch_acc > best_acc:
            best_acc = epoch_acc
            best_model_wts = model.state_dict()
            
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:2f}%'.format(best_acc*100))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

### Testing the model

For testing/validation only only forward pass is neccesary. Also we need to put the model in evaluation mode. This can be using .eval() with the model name.

In [ ]:
def test_model(model, phase='Test'):
    since = time.time()

    if phase == 'Test':
        print('-' * 10)

    # Set model to training mode
    model.eval()  
    running_corrects = 0

    # Iterate over data.
    for di, data in enumerate(valid_data_gen):
            # get the inputs
            inputs, labels = data

            # wrap them in Variable and convert to cuda
            if torch.cuda.is_available():
                inputs = Variable(inputs.cuda())
                labels = Variable(labels.cuda())
            else:
                inputs, labels = Variable(inputs), Variable(labels)

               
            # forward
            outputs = model(inputs)
            _, preds = torch.max(outputs.data, 1)

            # statistics
            running_corrects += torch.sum(preds.data == labels.data)

    test_acc = float(running_corrects) / dataset_valid

    print('{}: Acc: {:.2f}%'.format(phase, test_acc*100))
    print()

    if phase=='Test':
        time_elapsed = time.time() - since
        print('{} complete in {:.0f}m {:.0f}s'.format(phase, time_elapsed // 60, time_elapsed % 60))
    return test_acc

In [ ]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,  num_epochs=5)

### Thats all folks